In [22]:
import xarray as xr
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import cftime
import random
import time

In [8]:
from functools import wraps
from time import time

def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print(f'func:{f.__name__} took: {(te-ts):2.4f} sec')
        return result
    return wrap

In [9]:
def cftime_to_datetime64(date):
    """converting a cftime date into datetime64[D]
    
    Parameters:
    -----
    date: cftime
    """
    if type(date) is cftime._cftime.DatetimeNoLeap:
        return np.datetime64('{:0>4}'.format(str(date.year)) +'-'+ '{:0>2}'.format(str(date.month)) +'-'+ '{:0>2}'.format(str(date.day)),'D')
    if type(date) is list or type(date) is np.ndarray:
        output = []
        for t in date:
            output.append(np.datetime64('{:0>4}'.format(str(t.year)) +'-'+ '{:0>2}'.format(str(t.month)) +'-'+ '{:0>2}'.format(str(t.day)),'D'))
        return output

In [10]:
def is_in_doy_range_single(time, reference,day_range):
    """Checks if time is in a symmetric range of a reference date
    
    Parameters
    -----
    time: numpy.datetime64
    reference: numpy.datetime64
    day_range: int
    
    Returns
    -----
    True or False
    """

    time = time.astype(datetime)


    reference_max = (reference + day_range).astype(datetime)
    reference_min = (reference - day_range).astype(datetime)

    doy_time = time.timetuple().tm_yday
    doy_reference_max = reference_max.timetuple().tm_yday
    doy_reference_min = reference_min.timetuple().tm_yday

    if doy_reference_min < doy_reference_max:
        return doy_time in range(doy_reference_min,doy_reference_max)
    if doy_reference_min > doy_reference_max:
        return doy_time not in range(doy_reference_max, doy_reference_min)
    return None

In [25]:
def temporal_resampling(input_data, length,block_size,day_range):
    strt_time = time.time()
    """Temporal resampling of spatiotemporal input data with samples of given blocksize drawn from a range around the day of year of each sample

    Dates in the resampled dataset start with the 1000-01-01 for compatibility with most datetime formats downstream.
    Cropping the spatiotemporal input data to just the desired region is advised for better performance.

    Parameters:
    -----
    input_data: dict of xarray.core.dataset.Dataset
        spatiotemporal input data, should already be cropped to the region of interest for good performance
    length: int
        length in years of the resampled timeseries
    block_size: int
        size of the blocks for resampling the data in days. For 1 each day will be resampled individually
    day_range: int
        number of days of year before and after which samples can be drawn from
    """
    # dates = np.arange('1000-01-01', str(1000+length) + '-01-01', dtype='datetime64[D]')
    dates = xr.cftime_range(start = '{:0>4}'.format(str(1000)), end = '{:0>4}'.format(str(1000+ length)), freq='D',calendar="noleap")
    print(time.time()-strt_time)
    # the resulting samples will be stored in an dictionary with the same keys as the input dictionary
    # for each key (= variable), the samples will be stored in a list of xarray datasets
    samples = {}
    for key in input_data:
        samples[key] = []
    print(time.time()-strt_time)
    for i in range(0,dates.shape[0], block_size):
        reference_date = cftime_to_datetime64(dates[i])
        #select date from which the sample is drawn here so that it is the same for all variables
        possible_resampling_dates = [t for t in input_data[list(input_data.keys())[0]].time.values if is_in_doy_range_single(t.astype('datetime64[D]'),reference_date.astype('datetime64[D]'), day_range)]
        resampling_date = random.choice(possible_resampling_dates)
        for key in input_data:
            data = input_data[key]
            #get the sample at the selected date for every variable
            sample = data.sel(time = slice(resampling_date.astype('datetime64[D]'), resampling_date.astype('datetime64[D]') + block_size - 1))
            #overwrite the time variable with the date in the resampled data
            # sample = sample.assign_coords(time = np.arange(reference_date.astype('datetime64[D]'), reference_date.astype('datetime64[D]') + block_size,dtype='datetime64[D]'))
            sample = sample.assign_coords(time = xr.cftime_range(dates[i], dates[i]+pd.Timedelta(block_size - 1,'D'),freq='D',calendar="noleap" ))
            
            samples[key].append(sample)
        print(i, time.time()-strt_time)
    # merge the list of xarray datasets into a single one for each variable
    # then clip the time so that the timeseries is exactly the requested size
    for key in samples:
        samples[key] = xr.concat(samples[key],'time')
        #-1 as arrays in python start at 0, another -1 because xr.cftime_range include the end date (fist day of the following year) that we don't want
        samples[key] = samples[key].sel(time = slice(dates[0],dates[len(dates) - 2]))
    return samples

In [26]:
def create_xarray_array():
    data = np.random.rand(365*100, 3, 3)
    lat = ["A", "B", "C"]
    lon = ["D", "E", "F"]
    times = pd.date_range("2000-01-01", periods=365*100)
    
    return xr.DataArray(data, coords=[times, lon, lat], dims=["time", "lon", "lat"])


d1 = create_xarray_array()
d2 = create_xarray_array()

ds = {"v1":d1, "v2":d2}


In [27]:
resampled = temporal_resampling(input_data = ds,length = 2,block_size =10, day_range = 20)

0.0026793479919433594
0.002718687057495117
0.5387191772460938
1.096721887588501
1.6264169216156006
2.1307477951049805
2.6336848735809326
3.155487060546875
3.6746485233306885
4.224005460739136
4.7595436573028564
5.289365530014038
5.815620183944702
6.339022159576416
6.854413032531738
7.407238960266113
7.9238739013671875
8.441747665405273
8.94137167930603
9.432208061218262
9.927123308181763
10.417320489883423
10.913380146026611
11.415900468826294
11.915865659713745
12.406394481658936
12.89735722541809
13.398726463317871
13.908831596374512
14.454302549362183
14.966531038284302
15.476618766784668
15.982514142990112
16.49166750907898
17.00293254852295
17.520163536071777
18.037400722503662
18.553720474243164
19.0688579082489
19.57499670982361
20.089493989944458
20.60432457923889
21.114115953445435
21.60800337791443
22.10297727584839
22.602726221084595
23.10869002342224
23.607749938964844
24.106061220169067
24.608056783676147
25.104772090911865
25.606059789657593
26.107515573501587
26.62037467

In [48]:
def new_temporal_resampling(xarray_arr ,block_size =10, day_range = 20 ):
    xarray_arr = xarray_arr.sortby("time")
    
    for i in range(0,365,10):
        
    

In [49]:
new_temporal_resampling(d1)

[]


In [51]:
d1["time"].values[-1].astype('datetime64[Y]').astype(int)

129

In [55]:
d1["time"].values[-1].astype('datetime64[Y]')

numpy.datetime64('2099')